In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyedflib 
import pandas as pd
import mne
import sys
import eeg_lib as el

In [3]:
all_meseurments, all_targets = el.all_exp_to_array(4, [4])

Extracting EDF parameters from /home/daniel/repos/Decoding_of_EEG/files/S004/S004R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19679  =      0.000 ...   122.994 secs...


In [4]:
print(type(all_meseurments))
print(type(all_meseurments))
print(all_meseurments.shape)
print(all_targets.shape)




<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1, 20000, 64)
(1, 20000)


In [5]:
import torch
import torch.nn as nn


In [6]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out


In [44]:
one_probe= all_meseurments[0][0]
one_probe_T= all_targets

In [56]:
one_probe_T[0]

array([ 0.,  0.,  0., ..., nan, nan, nan])

In [57]:
x=np.array((one_probe_T[0],one_probe[1000-640:1000]))


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [54]:
batch= np.zeros((640,64))
dataset = np.zeros((20000,1,640,64))
conteiner= np.zeros((640,64))

for i in range(0,len(one_probe)):
    if i > 640:
        dataset[i][0]= np.array((one_probe_T[0],one_probe[i-640:i]))


In [53]:
dataset[300][0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
y_dataset= np.zeros((20000,640))
for i in range(0,len(one_probe_T)):
    if i > 640:
        y_dataset[i]= one_probe_T[i-640:i]

In [ ]:
dataset.shpa

In [23]:
y=dataset[:,:,:,:]
y.shape

IndexError: too many indices for array: array is 3-dimensional, but 4 were indexed

In [26]:
import torch.utils.data as 


IndexError: arrays used as indices must be of integer (or boolean) type

In [11]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import random_split
#spit data
X_train, X_test, y_train, y_test = train_test_split(dataset, y_dataset, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
# Convert the data to PyTorch tensors
X = torch.tensor(dataset)
y = torch.tensor(y_dataset)

# Define the train and test dataset sizes
train_size = int(0.8 * len(X))
test_size = len(X) - train_size

# Split the data into train and test datasets
train_dataset, test_dataset = random_split(list(zip(X, y)), [train_size, test_size])

# Access the train and test data
X_train, y_train = zip(*train_dataset)
X_test, y_test = zip(*test_dataset)

# Convert the train and test data back to tensors
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (16000, 640, 64)
Shape of X_test: (4000, 640, 64)
Shape of y_train: (16000, 640)


ValueError: only one element tensors can be converted to Python scalars

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU device is available")
else:
    device = torch.device("cpu")
    print("GPU device is not available, using CPU")


GPU device is available


In [ ]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)


AttributeError: 'numpy.ndarray' object has no attribute 'to'

In [ ]:
# Przygotowanie danych treningowych i testowych
# X_train, y_train - dane treningowe
# X_test, y_test - dane testowe

X_train= all_meseurments[0][0]
y_train

# Inicjalizacja modelu
model = LSTM(64, 64, 100,  1)

# Definicja funkcji straty i optymalizatora
criterion = nn.CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Pętla treningowa
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Ocena modelu na danych testowych
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)
        predictions = (test_outputs > 0.5).float()
        accuracy = (predictions == y_test).float().mean()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Accuracy: {accuracy.item():.4f}')


TypeError: 'int' object is not callable